# Convert Bannerlord Lore Model to GGUF

Run all cells in Google Colab (with GPU runtime) to convert your model to GGUF format for LM Studio/Ollama.

1. Open this notebook in Google Colab
2. Select Runtime -> Change runtime type -> T4 GPU
3. Run all cells
4. Download the generated GGUF file


In [ ]:
# Step 1: Install dependencies
!pip install unsloth huggingface_hub -q
print("Dependencies installed!")


In [ ]:
# Step 2: Download LoRA v2 adapter from HuggingFace
from huggingface_hub import snapshot_download

lora_id = "TSEOsiris/bannerlord-lore-lora-v2"
lora_dir = "./bannerlord-lore-lora"

print(f"Downloading LoRA adapter: {lora_id}...")
snapshot_download(repo_id=lora_id, local_dir=lora_dir)
print("LoRA adapter downloaded!")


In [ ]:
# Step 3: Load base model + LoRA adapter and convert to GGUF
import os
import gc
import torch
from unsloth import FastLanguageModel

os.makedirs("bannerlord-lore-gguf", exist_ok=True)

# Load the same clean base model used for training
base_model = "unsloth/Qwen3-8B"
print(f"Loading base model: {base_model}")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=base_model,
    max_seq_length=2048,
    load_in_4bit=True,
)
print("Base model loaded!")

# Load LoRA adapter
from peft import PeftModel
print("\nApplying LoRA adapter...")
model = PeftModel.from_pretrained(model, lora_dir)
print("LoRA adapter applied!")

# Free some memory
gc.collect()
torch.cuda.empty_cache()

# Save as GGUF Q4_K_M
print("\nConverting to GGUF Q4_K_M format...")
print("This may take 5-10 minutes...")

model.save_pretrained_gguf(
    "bannerlord-lore-gguf",
    tokenizer,
    quantization_method="q4_k_m"
)
print("\nGGUF conversion complete!")


In [ ]:
# Step 4: Download the GGUF file
import os
import glob
from google.colab import files

# List generated files
print("Generated files:")
for f in os.listdir("bannerlord-lore-gguf"):
    size = os.path.getsize(f"bannerlord-lore-gguf/{f}") / (1024**3)
    print(f"  {f}: {size:.2f} GB")

# Download GGUF file
gguf_files = glob.glob("bannerlord-lore-gguf/*.gguf")
if gguf_files:
    print(f"\nDownloading {gguf_files[0]}...")
    files.download(gguf_files[0])
else:
    print("No GGUF file found - check previous cell for errors")
